## Indonesia Public Company Financial Information Crawler
##### Last Update: 2022-08-02
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
Investing.com MAIN: https://www.investing.com /
Investing.com Indonesia: https://www.investing.com/equities/indonesia

Description: This is public company financial statement web crawling code for investing.com, especially for companies in Indonesia. investing.com is an unofficial stock information website for countries around the world. Even though it is an unofficial website, comparably, it has decent amount of information including key financial values. However, the server itself is very unstable that loading error occurs a lot, and data table format/language format is not standardized that a lot of exceptions are made(goes to failList). So one should be aware of that, and it is better to find alternative source for a long term development. Using excel format instead of csv could be a solution, which I have not tried so far. This code can be reused for other countries by editing initial website address and several page setting button XPATH locations - Colombia, Indonesia, Italy, Sinbgapore, and Spain is already written and uploaded. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [1]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

### 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - One also need to avoid ad and accept cookie banner that randomly appears.
#### - Run the crawler many times to improve the accuracy - Detail is written below main crawler.

### 2020

In [3]:
cover = open('Indonesia2020.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/indonesia')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(4)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(base_url + sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가정보
        info.append("IDN") 
        info.append("Indonesia") 
        info.append("Indonesia") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("275270000") 
        info.append("동남아시아") 

        #기업코드
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("IDNHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange. They are providing products/services in Indonesia, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Indonesia, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia, Asia"
                post = "N/A"
            except:
                address1 = "Indonesia"
                address2 = "Indonesia, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. USD ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. USD ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. USD ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. USD")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. USD")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. USD")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. USD")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. USD")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. USD")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. USD")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. USD")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. USD")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. USD")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. USD")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. USD")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. USD")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. USD")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. USD")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. USD")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. USD")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("IND")
        info.append("Indonesian")
        info.append("Bahasa Indonesia")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("IDX")
        info.append("Indonesia Stock Exchange")
        info.append("Indonesia Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "IDR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Indonesia")
        info.append("Indonesia, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("IDR")
        info.append("Indonesian Rupiah")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList.append(base_url + sub)
            
print(failList)

C:\Users\user\AppData\Local\Temp\ipykernel_19616\2280920221.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


<table class="genTbl closedTbl crossRatesTbl elpTbl elp25" id="cross_rate_markets_stocks_1" tablesorter=""><thead><tr><th class="flag"> </th><th class="left noWrap elp pointer">Name<span class="headerSortDefault newSiteIconsSprite" sort_default=""></span></th><th>Last</th><th>High</th><th>Low</th><th data-col-caption="Change">Chg.</th><th class="pointer" data-col-caption="Change %">Chg. %<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Vol">Vol.<span class="headerSortUp newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Time">Time<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="icon"></th></tr></thead><tbody><tr id="pair_101363"><td class="flag"><span class="ceFlags Indonesia" title="Indonesia"> </span></td><td class="bold left noWrap elp plusIconTd"><a href="/equities/bumi-resources" title="Bu

fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRAstrindo Nusantara Infrasruktur Tbk PT (BIPI)', 'Astrindo Nusantara Infrasruktur Tbk PT (BIPI)', 'Astrindo Nusantara Infrasruktur Tbk PT (BIPI)', "PT Astrindo Nusantara Infrastruktur Tbk is an Indonesia-based investment company. The Company operates its business activities in the mining, services, construction, and trading sectors. The Company focuses on integrated energy infrastructure that owns and operates mining infrastructure through its subsidiary. It conducts its business under two segments, namely Port Services, Mining and Other Services. Its Port services consist of the Company's operational activities at three ports, namely two ports in South Kalimantan and one port in East Kalimantan. The Company produces coal mining products through PT Putra Hulu Lematang (PHL). PHL has a mining production business license with an are

pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRBuana Listya Tama Tbk PT (BULL)', 'Buana Listya Tama Tbk PT (BULL)', 'Buana Listya Tama Tbk PT (BULL)', 'PT Buana Lintas Lautan Tbk is a shipping company. It provides a range of oil and gas tankers, designed to carry crude oil and oil products, as well as gas products, such as liquefied Petroleum Gas (LPG). It owns and operates VLGC (Very Large Gas Carrier). Its segments include Oil, Floating Production Storage and Offloading (FPSO) and Floating Storage and Offloading (FSO) tankers; Gas tankers; Chemical tankers, and Others. The Gas tankers segment provides maritime transportation of liquified gas, which includes LPG, propylene, propane and liquefied natural gas (LNG). The Chemical tankers segment provides maritime transportation of liquid chemical (organic and non-organic) and vegetable oil and animal fats. The Oil, FPSO and FSO tankers provide maritime transportation

pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRBintang Oto Global Tbk PT (BOGA)', 'Bintang Oto Global Tbk PT (BOGA)', 'Bintang Oto Global Tbk PT (BOGA)', 'PT Bintang Oto Global Tbk operates as an integrated automotive company in Indonesia. The company engages in the vehicle dealership and rental business; buying and sale of used cars; and provision of vehicle maintenance and repair services. It is also involved in the transportation, construction, industry, services, printing, workshop, agriculture, forestry, mining, and land transportation activities; and operation of information technology-based money lending service. PT Bintang Oto Global Tbk was founded in 2011 and is based in Malang, Indonesia.', 'Bintang Oto Global Tbk PT (BOGA)(English: Bintang Oto Global Tbk PT (BOGA)) is a public company that is listed on Indonesia Stock Exchange.', 'Bintang Oto Global Tbk PT (BOGA)(English: Bintang Oto Global Tbk PT (BOGA

...

fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRNipress Tbk (NIPS)', 'Nipress Tbk (NIPS)', 'Nipress Tbk (NIPS)', 'PT Nipress Tbk is an Indonesia-based company primarily engaged in manufacturing automotive and industrial batteries. The automotive batteries it manufactures include dry-charged batteries, marketed under the brand name NS Absolute; hybrid batteries, marketed under the brand name NS Accurate; maintenance-free batteries, marketed under the brand name NS Advance, and valve-regulated lead-acid (VRLA) batteries, marketed under the brand name NS Admiral. Its industrial batteries include: transportation power; motive power batteries, marketed under the brand name NS Accelerate, and stationary power, marketed under the brand name NS Accelerate. Its products are distributed in both domestic market and international markets, such as Latin America, Middle East, Africa, Asia a

pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRTrada Maritime Tbk (TRAM)', 'Trada Maritime Tbk (TRAM)', 'Trada Maritime Tbk (TRAM)', "PT Trada Alam Minera Tbk is an Indonesia-based company. The Company is engaged in the business of providing shipping and sea freight, mining services, development and general trading. The Company’s segments include shipping transportation, mining services and coal sales. Its business activities include floating storage and offloading (FSO) vessels, offshore supporting vessels for tugboats and barges, liquid cargo transportation, dry bulk carriers, and liquefied natural gas (LNG) carriers. It operates coal mining companies and mining service companies in Kabupaten Kutai Barat, East Kalimantan. The Company's subsidiaries include PT Semeru Infra Energy, PT SMR Utama Tbk, PT Hanochem Shipping, PT Black Diamond Energy, PT Bahari Sukses Utama, PT Trade Offshore Services, PT Agate Bumi Tank

pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRInti Kapuas Arowana Tbk (IIKP)', 'Inti Kapuas Arowana Tbk (IIKP)', 'Inti Kapuas Arowana Tbk (IIKP)', 'PT Inti Agri Resources Tbk is an Indonesia-based company that is engaged in business activities in Arowana fish breeding and trading. The Company breeds super red Arowana, which it markets under the brand name of ShelookRED, and green Arowana. The Company’s subsidiaries include PT Inti Kapuas International and PT Bahari Istana Alkausar, which are engaged in fish breeding Arowana.', 'Inti Kapuas Arowana Tbk (IIKP)(English: Inti Kapuas Arowana Tbk (IIKP)) is a public company that is listed on Indonesia Stock Exchange.', 'Inti Kapuas Arowana Tbk (IIKP)(English: Inti Kapuas Arowana Tbk (IIKP)) is a public company that is listed on Indonesia Stock Exchange.', None, None, '', '', '', '62 21 5835 7337', '62 21 5835 8042', '', 'www.shelookred.com', '', '', '', '', '', 'Puri Br

pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRCitra Putra Realty Tbk PT (CLAY)', 'Citra Putra Realty Tbk PT (CLAY)', 'Citra Putra Realty Tbk PT (CLAY)', 'PT Citra Putra Realty Tbk engages in the hospitality, real estate, and property businesses in Indonesia. The company purchases, sells, rents, and operates self-owned or leased real estate properties, such as apartment and residential buildings, exhibition venues, personal storage facilities, malls, and shopping centers. It also owns and operates hotels under The Stones Legian Hotel and Clay Hotel names. In addition, the company is involved in the wholesale of machinery, equipment, paint, varnish, and building supplies; and trade of rough wood, main processed wood products, and building materials, such as markets and gravel. Further, it provides consulting services in human resource, real estate property areas operation management and maintenance, property and ass

fail
pass
pass
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBROrganon Pharma (SCPI)', 'Organon Pharma (SCPI)', 'Organon Pharma (SCPI)', 'PT Organon Pharma Indonesia Tbk engages in manufacturing, packaging, and distributing pharmaceutical products in Indonesia. The company operates through four segments: Primary Care, Vaccines, Oncology & Specialities, and Alliance Management. The company provides medicines for dermatology, vaccine, neurology, anti-infective, cardiovascular, immunology, and allergical diseases, as well as oncology, diabetes, and hypertension. The company was founded in 1972 and is headquartered in Jakarta, Indonesia. PT Organon Pharma Indonesia Tbk operates as a subsidiary of Organon LLC.', 'Organon Pharma (SCPI)(English: Organon Pharma (SCPI)) is a public company that is listed on Indonesia Stock Exchange.', 'Organon Pharma (SCPI)(English: Organon Pharma (SCPI)) i

### 2019

In [ ]:
cover = open('Indonesia2019.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/indonesia')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(5)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(base_url + sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가정보
        info.append("IDN") 
        info.append("Indonesia") 
        info.append("Indonesia") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("275270000") 
        info.append("동남아시아") 

        #기업코드
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("IDNHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange. They are providing products/services in Indonesia, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Indonesia, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia, Asia"
                post = "N/A"
            except:
                address1 = "Indonesia"
                address2 = "Indonesia, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[4]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[4]').text + "M. USD ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. USD ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[4]').text + "M. USD ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. USD")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. USD")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. USD")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. USD")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. USD")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. USD")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. USD")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. USD")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. USD")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[4]').text + "M. USD")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. USD")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. USD")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. USD")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. USD")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. USD")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. USD")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. USD")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[4]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("IND")
        info.append("Indonesian")
        info.append("Bahasa Indonesia")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("IDX")
        info.append("Indonesia Stock Exchange")
        info.append("Indonesia Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "IDR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Indonesia")
        info.append("Indonesia, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("IDR")
        info.append("Indonesian Rupiah")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList.append(base_url + sub)
            
print(failList)

C:\Users\user\AppData\Local\Temp\ipykernel_19616\2202389824.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


<table class="genTbl closedTbl crossRatesTbl elpTbl elp25" id="cross_rate_markets_stocks_1" tablesorter=""><thead><tr><th class="flag"> </th><th class="left noWrap elp pointer">Name<span class="headerSortDefault newSiteIconsSprite" sort_default=""></span></th><th>Last</th><th>High</th><th>Low</th><th data-col-caption="Change">Chg.</th><th class="pointer" data-col-caption="Change %">Chg. %<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Vol">Vol.<span class="headerSortUp newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Time">Time<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="icon"></th></tr></thead><tbody><tr id="pair_101363"><td class="flag"><span class="ceFlags Indonesia" title="Indonesia"> </span></td><td class="bold left noWrap elp plusIconTd"><a href="/equities/bumi-resources" title="Bu

fail
pass
fail
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRAstrindo Nusantara Infrasruktur Tbk PT (BIPI)', 'Astrindo Nusantara Infrasruktur Tbk PT (BIPI)', 'Astrindo Nusantara Infrasruktur Tbk PT (BIPI)', "PT Astrindo Nusantara Infrastruktur Tbk is an Indonesia-based investment company. The Company operates its business activities in the mining, services, construction, and trading sectors. The Company focuses on integrated energy infrastructure that owns and operates mining infrastructure through its subsidiary. It conducts its business under two segments, namely Port Services, Mining and Other Services. Its Port services consist of the Company's operational activities at three ports, namely two ports in South Kalimantan and one port in East Kalimantan. The Company produces coal mining products through PT Putra Hulu Lematang (PHL). PHL has a mining production business license w

fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
fail
pass
fail
pass
pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRBumi Resources Minerals Tbk (BRMS)', 'Bumi Resources Minerals Tbk (BRMS)', 'Bumi Resources Minerals Tbk (BRMS)', 'PT Bumi Resources Minerals Tbk. (BRM) is an Indonesia-based multi-minerals mining company. The Company is focused on operating a diverse portfolio of minerals including copper, gold, zinc, and lead. The Company’s business units include PT Citra Palu Minerals (CPM), PT Dairi Prima Mineral (DPM) and PT Gorontalo Minerals (GM). CPM operates one gold ore processing plant and has contract of work rights to an approximately 85,180 hectare (ha) mining concession in Central Sulawesi and South Sulawesi, Indonesia. DPM operates portfolio of minerals zinc & lead. DPM has contract of work is located in North Sumatra and is proposed to be developed as an underground mining operation (mining 

pass
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['IDN', 'Indonesia', 'Indonesia', 'UTC+07:00', '아시아', '6100000000000 USD', '275270000', '동남아시아', 'IDNHBRSurya Citra Media Tbk (SCMA)', 'Surya Citra Media Tbk (SCMA)', 'Surya Citra Media Tbk (SCMA)', 'PT Surya Citra Media Tbk, together with its subsidiaries, provides multimedia services in Indonesia. It owns SCTV and Indosiar television channels. The company also provides entertainment and advertising, artist management, streaming video, and television broadcasting services. In addition, it engages in the film and content trading, content management and production, production house, and multimedia operations; and film production and video recording activities. Further, the company offers film distribution and marketing services; information technology, promotion and marketing, general telecommunication, management and production agency, public relations, and mass media and audio recording services; and rental and management consulta

fail
pass
fail
pass
pass
pass
pass


### 2018

In [ ]:
cover = open('Indonesia2018.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/indonesia')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(4)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(base_url + sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가정보
        info.append("IDN") 
        info.append("Indonesia") 
        info.append("Indonesia") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("275270000") 
        info.append("동남아시아") 

        #기업코드
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("IDNHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Indonesia Stock Exchange. They are providing products/services in Indonesia, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Indonesia, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Indonesia, Asia"
                post = "N/A"
            except:
                address1 = "Indonesia"
                address2 = "Indonesia, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[5]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[5]').text + "M. USD ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. USD ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[5]').text + "M. USD ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. USD")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. USD")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. USD")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. USD")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. USD")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. USD")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. USD")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. USD")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. USD")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[5]').text + "M. USD")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. USD")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. USD")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. USD")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. USD")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. USD")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. USD")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. USD")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[5]').text + "M. USD")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. USD")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("IND")
        info.append("Indonesian")
        info.append("Bahasa Indonesia")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("IDX")
        info.append("Indonesia Stock Exchange")
        info.append("Indonesia Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "IDR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Indonesia")
        info.append("Indonesia, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("IDR")
        info.append("Indonesian Rupiah")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList.append(base_url + sub)
            
print(failList)

### 4. Dealing with FailList
#### - Because Investing.com causes loading time error due to unstable server status, it is better to run the code, at least with companies in failList, for several times or run with two or more computers. That will increase the sucess rate much higher than running the code once. When running a new code, make sure to change the name of saving file so that you don't replace the original file.
#### - There are many ways to deal with elements failList: retry(if caused by server loading status), ignore(if caused by actually non-existing data), manually add(if only few exist), find why error has occurred(if massive).
#### - Below is both actual and sample code that was used in 2022

In [ ]:
print(len(failList))

In [5]:
cover = open('Indonesia_2022_investing_2022-07-04_Chris - 2.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791